In [23]:
from datetime import datetime

from dscreator.cfarray.base import dataarraybytime, DEFAULT_ENCODING
from dscreator.cfarray.trajectory import trajectorycoords, trajectorydataset


In [24]:

longitudes = [10.70, 10.60, 10.50, 10.40]
latitudes = [50.70, 50.60, 50.50, 50.40]
time = list(
    map(
        datetime.fromisoformat,
        [
            "1970-06-01T00:00:00",
            "1970-06-01T10:00:00",
            "1980-06-01T10:00:00",
            "1990-06-01T10:00:00",
        ],
    )
)



In [25]:
temperature = dataarraybytime(
    name="sea_water_temperature",
    standard_name="sea_water_temperature",
    long_name="sea_water_temperature",
    units="degree_Celsius",
    data=[2, None, 5, 7],
).assign_coords(
    trajectorycoords(
        time=time,
        latitude=latitudes,
        longitude=longitudes,
    )
)

In [26]:
turbidity = dataarraybytime(
    name="sea_water_turbidity",
    standard_name="sea_water_turbidity",
    long_name="sea_water_turbidity",
    units="degree_Celsius",
    data=[None, 1, 5, 7],
).assign_coords(
    trajectorycoords(
        time=time,
        latitude=latitudes,
        longitude=longitudes,
    )
)

In [27]:
ds = trajectorydataset([temperature, turbidity], "trajectory_name")


In [28]:
ds

<xarray.Dataset>
Dimensions:                (time: 4)
Coordinates:
  * time                   (time) datetime64[ns] 1970-06-01 ... 1990-06-01T10...
    longitude              (time) float64 10.7 10.6 10.5 10.4
    latitude               (time) float64 50.7 50.6 50.5 50.4
Data variables:
    sea_water_temperature  (time) object 2 None 5 7
    sea_water_turbidity    (time) object None 1 5 7
    trajectory_name        <U15 'trajectory_name'
Attributes:
    featureType:  trajectory

In [29]:
ds.to_netcdf("example_trajectory.nc", encoding=DEFAULT_ENCODING)
# run `ncdump example_trajectory.nc` on commandline to view text repr also


In [ ]:

# Example TrajectoryExtractor
# also see tests/test_trajectories.py

In [30]:
from datetime import datetime

from dscreator.datasets import trajectories
from dscreator.sources.base import NamedTrajectory, Point



In [45]:
# Example input to the trajectory builder
trajectory = NamedTrajectory(
    "Temp",
    locations=[Point(10.71, 50.70), Point(10.70, 50.71), Point(10.70, 50.72)],
    values=[1, 2, 3],
    datetime_list=[datetime(1999, 10, 4), datetime(1999, 10, 5), datetime(1999, 10, 6)],
)

# For adding other arrays they will currently need to share locations! datetime_list
# can be different, but it is not recommended

turbidity = NamedTrajectory(
    "Turbidity",
    locations=[Point(10.71, 50.70), Point(10.70, 50.71), Point(10.70, 50.72)],
    values=[1, None, 4],
    datetime_list=[datetime(2000, 10, 4), datetime(2000, 10, 5), datetime(2000, 10, 6)],
)




In [46]:
example_builder = trajectories.example.ExampleTrajBuilder(
    "uuid", "dataset_name", "trajectory_name", "project_name", True
)

In [47]:
ds = example_builder.create([trajectory, turbidity])

In [48]:
ds


<xarray.Dataset>
Dimensions:          (time: 6)
Coordinates:
  * time             (time) datetime64[ns] 1999-10-04 1999-10-05 ... 2000-10-06
    longitude        (time) float64 10.71 10.7 10.7 10.71 10.7 10.7
    latitude         (time) float64 50.7 50.71 50.72 50.7 50.71 50.72
Data variables:
    temperature      (time) float64 1.0 2.0 3.0 nan nan nan
    turbidity        (time) object nan nan nan 1 None 4
    trajectory_name  <U15 'trajectory_name'
Attributes: (12/28)
    featureType:          trajectory
    id:                   uuid
    title:                example
    summary:              summary
    date_created:         2023-02-06 16:58:02.620486
    keywords:             
    ...                   ...
    netcdf_version:       4
    publisher_name:       Norwegian Institute for Water Research
    publisher_email:      miljoinformatikk@niva.no
    publisher_url:        https://niva.no
    licence:              CC-BY-4.0
    history:              Initial data